In [9]:
# import sys;
# print(sys.version)

In [15]:
import boto3
import os

class KinesisStream(object):

    CLIENT = boto3.client('kinesis')
    SHARD_COUNT = 2

    def __init__(self, stream_name):
        self.stream_name = stream_name

    def list(self):
        return self.CLIENT.list_streams()['StreamNames']

    def create(self):
        self.CLIENT.create_stream(
            StreamName=self.stream_name,
            ShardCount=self.SHARD_COUNT
            )

    def add_tags(self):
        self.CLIENT.add_tags_to_stream(
            StreamName=self.stream_name,
            Tags=self.build_tags()
            )

    def build_tags(self):
        return {
            'BUSINESS_REGION': 'NORTHAMERICA',
            'BUSINESS_UNIT': 'DATASERVICES',
            'CLIENT': 'NONE',
            'ENVIRONMENT': 'POC',
            'NAME': self.stream_name,
            'PLATFORM': 'ATLAS'
        }

    def get_arn(self):
        return self.CLIENT.describe_stream(
            StreamName=self.stream_name
            )['StreamDescription']['StreamARN']

class KinesisFirehose(object):
    CLIENT = boto3.client('firehose')
    iam = boto3.client('iam')

    def __init__(self, firehose_name, bucket_name, prefix_name):
        self.firehose_name = firehose_name
        self.bucket_name = bucket_name
        self.prefix_name = prefix_name

    def list(self):
        return self.CLIENT.list_delivery_streams()['DeliveryStreamNames']

    def create(self):
        print("********************")
        print(self.bucket_name)
        return self.CLIENT.create_delivery_stream(
            DeliveryStreamName=self.firehose_name,
            S3DestinationConfiguration={'BucketARN': "arn:aws:s3:::"+self.bucket_name,
                                        'RoleARN': 'arn:aws:iam::714861692883:role/firehose_delivery_role'}
            )
    
    def create_role(name,policies=None):
        """ Create a role with an optional inline policy """
        policydoc = {
            "Version": "2012-10-17",
            "Statement": [
                {"Effect": "Allow", "Principal": {"Service": ["lambda.amazonaws.com"]}, "Action": ["sts:AssumeRole"]},
            ]
        }
        roles = [r['RoleName'] for r in iam.list_roles()['Roles']]
        if name in roles:
            print('IAM role %s exists' % (name))
            role = iam.get_role(RoleName=name)['Role']
        else:
            print('Creating IAM role %s' % (name))
            role = iam.create_role(RoleName=name, AssumeRolePolicyDocument=json.dumps(policydoc))['Role']

        # attach managed policy
        if policies is not None:
            for p in policies:
                iam.attach_role_policy(RoleName=role['RoleName'], PolicyArn=p)
        return role
    
    
    def config(self):
        role=self.create_role('lambda', policies=["AWSLambdaKinesisExecutionRole"])
        return {
            'RoleARN': role,
            'BucketARN': 'arn:aws:s3:::' + self.bucket_name,
            'Prefix': self.prefix_name,
            'BufferingHints': {
                'SizeInMBs': 128,
                'IntervalInSeconds': 900
            },
            'CompressionFormat': 'Snappy',
            'EncryptionConfiguration': {
                'NoEncryptionConfig': 'NoEncryption'
            },
            'CloudWatchLoggingOptions': {
                'Enabled': True,
                'LogGroupName': '/aws/kinesisfirehose/' + self.firehose_name,
                'LogStreamName': 'S3Delivery'
            }
        }


class S3(object):
    RESOURCE = boto3.resource('s3')
    CONFIG_FILE_BUCKET = 'dsabucket1'
    CONFIG_FILE_PREFIX = 'lambda-configs/'

    def __init__(self, config_file):
        self.file = config_file

    def upload_file_to_config_folder(self):
        print(self.file)
        self.RESOURCE.meta.client.upload_file(
            os.path.realpath(self.file),
            self.CONFIG_FILE_BUCKET, 
            self.CONFIG_FILE_PREFIX + self.file
            )

class Lambda(object):

    CLIENT = boto3.client('lambda')
    
    
    def __init__(self, stream_arn, function_name):
        self.stream_arn = stream_arn
        self.function_name = function_name

    def event_source_list(self):
        return self.CLIENT.list_event_source_mappings(
            EventSourceArn=self.stream_arn, 
            FunctionName=self.function_name
            )['EventSourceMappings']

    def create_event_source(self,func_name, kinesis_stream_arn, start_pos):
        print("event source is creating")
        return self.CLIENT.create_event_source_mapping(
            EventSourceArn=kinesis_stream_arn,
            FunctionName=func_name,
            Enabled=True,
            BatchSize=100,
            StartingPosition=start_pos
            )
    
    def create_role(name, policies=None):
        """ Create a role with an optional inline policy """
        policydoc = {
            "Version": "2012-10-17",
            "Statement": [
                {"Effect": "Allow", "Principal": {"Service": ["lambda.amazonaws.com"]}, "Action": ["sts:AssumeRole"]},
            ]
        }
        roles = [r['RoleName'] for r in iam.list_roles()['Roles']]
        if name in roles:
            print('IAM role %s exists' % (name))
            role = iam.get_role(RoleName=name)['Role']
        else:
            print('Creating IAM role %s' % (name))
            role = iam.create_role(RoleName=name, AssumeRolePolicyDocument=json.dumps(policydoc))['Role']

        # attach managed policy
        if policies is not None:
            for p in policies:
                iam.attach_role_policy(RoleName=role['RoleName'], PolicyArn=p)
        return role

    
    def create_function(self,name, zfile, role, lsize=512, timeout=10, update=False):
        """ Create, or update if exists, lambda function """
        with open(zfile, 'rb') as zipfile:
            if name in [f['FunctionName'] for f in lamb.list_functions()['Functions']]:
                if update:
                    print('Updating %s lambda function code' % (name))
                    return lamb.update_function_code(FunctionName=name, ZipFile=zipfile.read())
                else:
                    print('Lambda function %s exists' % (name))
                    for f in funcs:
                        if f['FunctionName'] == name:
                            lfunc = f
            else:
                print('Creating %s lambda function' % (name))
                lfunc = lamb.create_function(
                    FunctionName=name,
                    Runtime='python3.6',
                    Role=role['Arn'],
                    Handler='lambda.lambda_handler',
                    Description='Example lambda function to ingest a Kinesis stream',
                    Timeout=timeout,
                    MemorySize=lsize,
                    Publish=True,
                    Code={'ZipFile': zipfile.read()},
                )
            lfunc['Role'] = role
            return lfunc

In [11]:
import os
import zipfile

zf = zipfile.ZipFile("lambda_test.zip", "w")
zf.write('lambda_test.py')
zf.close()

In [23]:
iam_client = boto3.client('iam')
    
lambda_policy_for_role = iam_client.create_policy(
PolicyName='testing_lambda-assumerole',
Description="This policy allows the Lambda function to write data to kinesis stream",
PolicyDocument="""{
                    "Version": "2012-10-17",
                    "Statement": [
                                    {
                                        "Effect": "Allow",
                                        "Action": [
                                            "lambda:InvokeFunction"
                                        ],
                                        "Resource": [
                                            "*"
                                        ]
                                    },
                                    {
                                        "Effect": "Allow",
                                        "Action": [
                                            "kinesis:GetRecords",
                                            "kinesis:GetShardIterator",
                                            "kinesis:DescribeStream",
                                            "kinesis:ListStreams",
                                            "logs:CreateLogGroup",
                                            "logs:CreateLogStream",
                                            "logs:PutLogEvents"
                                        ],
                                        "Resource": "*"
                                    }
                                ]  }"""
)

In [26]:
lambda_policy_for_role['Policy']['Arn']

'arn:aws:iam::714861692883:policy/testing_lambda-assumerole'

In [25]:
import sys
import json
# import aws_pipeline


LAMBDA_FUNC_EXCEPTION = 'EXTERNALCONFIG'

def get_config_file():
    '''
    Gets the config file from the argument
    '''

    try:
        return "config.json"
    except IndexError:
        print("Error: config file is missing - please add.")
        sys.exit()

def read_config_file(config_file):
    '''
    Reads and returns the JSON object
    '''

    with open(config_file) as config:
        data = json.load(config)

    return data


def main():
    '''
    the main thang
    '''

    # get, read config file and make JSON accessible
    config_file = get_config_file()
    data = read_config_file(config_file)

    # set various values
    stream_name = data['stream']
    firehose_name = data['firehose']
    bucket_name = data['bucket']
    prefix_name = data['prefix']

    lambda_function_name=""
    
    print(config_file)
    s_three = S3(config_file)
    stream = KinesisStream(stream_name)
    firehose = KinesisFirehose(firehose_name, bucket_name, prefix_name)

    # upload config file to s3
    s_three.upload_file_to_config_folder()

    # check if stream exists, if not create
    if stream_name not in stream.list():
        stream.create()
        stream.add_tags()
    else:
        print('STATUS: Stream found - do nothing')

    # check if firehose exists, if not create
    if firehose_name not in firehose.list():
        firehose.create()
    else:
        print('STATUS: Firehose found - do nothing')

    stream_arn = stream.get_arn()
    lambda_exception_handler = Lambda(stream_arn, LAMBDA_FUNC_EXCEPTION)

    
    lambda_role=lambda_exception_handler.create_role('LambdaKinesisDynamo-role', policies=[lambda_policy_for_role['Policy']['Arn']])
    
    lfunc = lambda_exception_handler.create_function(stream_arn, 'lambda_test.zip', role=lambda_role, update=True)
        
    # check if lambda event mapping exits for exception handler function, if not create
    
    sources = lambda_exception_handler.event_source_list()
    print("sources:",sources)
    print("stream:",stream_arn)
    if stream_arn not in [s['EventSourceArn'] for s in sources]:
        source = lambda_exception_handler.create_event_source(func_name="lambda_test", kinesis_stream_arn=stream_arn,start_pos='TRIM_HORIZON')
    else:
        print('STATUS: Lambda event source found - do nothing')




#     if not lambda_exception_handler.event_source_list():
#         lambda_function_name=lambda_exception_handler.create_function(name, zfile)
#         lambda_function_name
# #         lambda_exception_handler.create_event_source()
#     else:
#         print('STATUS: Lambda event source found - do nothing')


main()

config.json
config.json
STATUS: Stream found - do nothing
STATUS: Firehose found - do nothing


TypeError: create_role() got multiple values for argument 'policies'